In [1]:
import sys
sys.path.insert(0, "..")
import json

from src.db_ingestion.chroma_client import query_to_collection
from src.talent_selection_flow.crews.cv_to_job_crew.crew import CVToJobCrew

from src.db_ingestion.metadata_extraction_crew.crews import CVMetadataExtractorCrew
from src.db_ingestion.metadata_extraction_crew.enums import ExperienceLevel, EducationLevel

In [2]:
cv_example = "Experienced Software Engineer with 5+ years in Python, specialized in building scalable AI agents and distributed systems using CrewAI and FastAPI. From Houston"
top_k = 3